In [1]:
import os,re,glob
import pandas as pd
import numpy as np
from collections import Counter
from rna_seq_normalization import Normalization as Norm
from functools import reduce

users_dir = "/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction"
specie = "human"
# GENOMEDIR = "/genomics/users/marta/genomes"
GENOMEDIR = "/data/genomics/marta/genomes"

### cancer data
cancer_dir = users_dir + "/cancers"
raw_cancer_dir="/users/genomics/marta/TCGA_RNASeq"
tcga_projects=["TCGA-BRCA","TCGA-LUSC","TCGA-PRAD","TCGA-KIRC","TCGA-KIRP","TCGA-LUAD","TCGA-BLCA"]#,"TCGA-LIHC"]
other_projects=["GSE102101_KIRC","GSE133624_BLCA","GSE22260_PRAD","PRJEB2449_PRAD","SRP238334_KIRC","GSE214846_LIHC","GSE229705_LUAD","TCGA_COAD","SRP107326_COAD"]
# deleted=["GSE103001_BRCA","GSE89223_PRAD"]
manuscript_projects = ["liver_adjacent_totalRNA_LIHC","hcc_normal_totalRNA_LIHC","GSE193567_LIHC","LIHC_TCGA_LIHC"]
all_projects = tcga_projects + other_projects + manuscript_projects

cancertypes = ["BRCA","BLCA","LUAD","KIRC","PRAD","LUSC","LIHC","COAD"]
## annotation file
annotation="/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/gencode.v38.gffcompare.TestisLiverBrain.annotation.sorted.1transcript.sorted.NOchr.gtf"
transcript_gene=pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/newReference_Resconstructed/1transcript_1gene.reconstructed.csv")

def count_greater_than_one(row):
    return (row > 1).sum()

In [5]:
candidates=pd.read_csv(os.path.join(cancer_dir,"log2ratio3x/cancertypes/TOv3x_5percent_TestisRestrictedGTEx_Translated_Ctypes_log2ratio3xMEANgenes.csv"))
candidates

,Unnamed: 0,gene_name,gene_id,transcript_id,gene_type,Length,num_patients_overexpr,percentage_num_patients_overexpr,ctype
0,0,MAGEC2,ENSG00000046774,ENST00000247452,protein_coding,1994.0,8.0,7.34,BRCA
1,4,ACTL8,ENSG00000117148,ENST00000375406,protein_coding,1841.0,27.0,24.77,BRCA
2,7,MAGEA10,ENSG00000124260,ENST00000244096,protein_coding,2741.0,6.0,5.50,BRCA
3,9,SSX1,ENSG00000126752,ENST00000376919,protein_coding,1223.0,6.0,5.50,BRCA
4,11,LIN28A,ENSG00000131914,ENST00000326279,protein_coding,3975.0,7.0,6.42,BRCA
...,...,...,...,...,...,...,...,...,...
257,77,LINC02864,ENSG00000263711,ENST00000581862,lncRNA,3634.0,13.0,9.03,COAD
258,86,XLOC_000310,XLOC_000310,TCONS_00000389,novel,322.0,10.0,6.94,COAD
259,88,XLOC_000930,XLOC_000930,TCONS_00001039,novel,6255.0,55.0,38.19,COAD
260,91,XLOC_001049,XLOC_001049,TCONS_00001208,novel,2720.0,29.0,20.14,COAD


In [6]:
human_seqs_riboseq_testis = pd.read_csv("/users/genomics/marta/TestisProject_SaraRazquin/with_TranscriptomeReconstruction/human/240724_RiboNovel/RibORF/repre.valid.pred.pvalue.parameters.allsamples.noredundant.PROTEIN.fa", header=None, sep="\t")
df = pd.DataFrame({'header':human_seqs_riboseq_testis[0].iloc[::2].values, 'seq':human_seqs_riboseq_testis[0].iloc[1::2].values})
df['transcript_id'] = df['header'].str.split(":", expand=True)[0]
df['transcript_id'] = df['transcript_id'].str.split("\.", expand=True)[0]
df['transcript_id'] = df['transcript_id'].str.split(">", expand=True)[1]
df['ORFtype'] = df['header'].str.split("\|", expand=True)[3]
df


,header,seq,transcript_id,ORFtype
0,>ENST00000381163.7:X:+|14|3393:283:1789|canoni...,MSETEFHHGAQAGLELLRSSNSPTSASQSAGMTVTDQAFVTLATND...,ENST00000381163,canonical
1,>ENST00000381154.6:X:-|4|5145:65:1847|canonica...,MRSAARRGRAAPAARDSLPVLLFLCLLLKTCEPKTANAFKPNILLI...,ENST00000381154,canonical
2,>ENST00000683290.1:X:-|31|2976:345:2190|canoni...,MRPVINRCAPGSLDLMLPQAASEGIVFHSLQISLCFRSWLPAMLAV...,ENST00000683290,canonical
3,>ENST00000381127.6:X:+|19|2165:222:1995|canoni...,MRPRRPLVFMSLVCALLNTCQAHRVHDDKPNIVLIMVDDLGIGDLG...,ENST00000381127,canonical
4,>ENST00000217939.7:X:-|6|9804:92:8654|extensio...,MLPAHRSRVRRLPKDRYPRIRHPDKMPKRAHWGALSVVLILLWGHP...,ENST00000217939,extension
...,...,...,...,...
43012,>ENST00000468564.6:22:+|19|5066:195:372|noncod...,VLKSVGDLQGVWPSAPQGPHALVIPTLLPGGKKKYLPPTSRKDPKF...,ENST00000468564,noncoding
43013,>ENST00000468564.6:22:+|312|5066:3913:3970|non...,MRAHGTSLMNYLSWLRRK*,ENST00000468564,noncoding
43014,>ENST00000381176.5:22:-|350|6853:4262:4343|dOR...,MSSFMLSLSSVYGVIVVCFRISIQNQ*,ENST00000381176,dORF
43015,>ENST00000608025.2:22:-|4|1001:41:164|noncodin...,MGTPPRPPATTPLGQPRISSAAPAPAAGTEARVRSGRAGA*,ENST00000608025,noncoding


In [15]:
ORFs_from_candidates = df.merge(candidates, on="transcript_id")
set(ORFs_from_candidates[(ORFs_from_candidates['gene_type'] == "protein_coding") & (ORFs_from_candidates['ORFtype'] != "canonical")].gene_name.values.tolist())

{'ACTL8',
 'ADAD1',
 'CBY2',
 'COX7B2',
 'DCAF4L2',
 'DCAF8L2',
 'GOLGA6L7',
 'KCNU1',
 'LUZP4',
 'LYZL2',
 'MAGEA10',
 'MAGEB1',
 'MAGEC1',
 'MAGEC2',
 'MEIOB',
 'PAGE2',
 'PRSS54',
 'RBM46',
 'RFPL4B',
 'RNF17',
 'SOX30',
 'SPATA12',
 'SSX3',
 'TFDP3',
 'TSBP1',
 'VCX',
 'ZFP42'}